# 実用的なPEGパーサの開発

## 11.1 パーザコンテクスト

In [1]:
class PContext(object):
    inputs: str # 解析対象の入力文字列
    pos: int # パーザが現在解析している位置
    epos: int # 文字列上の終端の位置
    errpos: int # エラーが発生した位置
    ptree: object # パーザが構築中の構文木
    memos: list # Packrat 構文解析法用のメモ表


In [2]:
def pChar(text):
    def match(px):
        if px.inputs.startswith(text, px.pos):
            px.pos += len(text)
            return True
        return False
    return match

pChar('abc')

<function __main__.pChar.<locals>.match(px)>

In [3]:
 def pSeq(e1, e2):
    def match(px):
        return e1(px) and e2(px)
    return match

In [5]:
def pOre2(e1, e2):
    def match(px):
        pos = px.pos # ローカル変数にコピー 
        if e1(px):
            return True
        px.errpos = max(px.pos, px.errpos)
        px.pos = pos # ローカル変数からバックトラッキング処理 
        return e2(px)
    return match

In [6]:
def pNot(e):
    def match(px):
        pos = px.pos # 処理前の状態をローカル変数に 
        if not e():
            px.errpos = max(px.pos, px.errpos) 
            px.pos = pos # バックトラッキング処理 
            return True
        return False
    return match

In [7]:
def pMany(e):
    def match(px):
        pos = px.pos
        while e(px) and px.pos > pos:
            pos = px.pos # 状態を更新 
        px.errpos = max(px.pos, px.errpos) 
        return True
    return match

## 構文木

In [8]:
def pToken(e):
    def match(px):
        pos = px.pos
        if e(px):
            token = px.inputs[pos:px.pos]
            px.ptree.append(token)
            return True
        return False
    return match

In [9]:
def pToken(e):
    def match(px):
        pos = px.pos
        if e(px):
            token = px.inputs[pos:px.pos]
            px.ptree = px.tree + (token,)
            return True
        return False
    return match

In [10]:
def pToken(e):
    def match(px):
        pos = px.pos
        if e(px):
            token = px.inputs[pos:px.pos]
            px.ptree = (px.ptree, token)
            return True
        return False
    return match

In [11]:
def pRange(chars, ranges):
    def match(px):
        if px.pos < px.epos:
            c = px.inputs[px.pos]
            for c2 in chars:
                if c == c2:
                    px.pos += 1
                    return True
            rs = ranges
            while len(rs) > 0:
                if rs[0] <= c <= rs[1]:
                    px.pos += 1
                    return True
                rs = rs[2:]
        return False
    return match

In [ ]:
# ライブラリを活用しない場合
c = px.inputs[px.pos]
for c2 in chars:
    if c == c2:
        px.pos += 1
        return True

# ライブラリを活用する場合
c = px.inputs[px.pos]
if chars.find(c) != -1:
    px.pos += 1
    return True

In [12]:
def tochars(chars, ranges):
    cs = set(list(chars))
    rs = ranges
    while len(rs) > 0:
       r = range(ord(rs[0]), ord(rs[1])+1)
       cs |= set(map(chr, r))
       rs = rs[2:]
    return ''.join(cs)

In [ ]:
# 事前に変換する
def pRange(chars, ranges):
    chars = tochars(chars, ranges)
    def match(px):
        if px.pos < px.epos:
            c = px.inputs[px.pos]
            if chars.find(c) != -1:
                px.pos += 1
                return True
        return False
    return match

In [ ]:
# パーザ関数内で変換する
def pRange(chars, ranges):
    def match(px):
        if px.pos < px.epos:
            c = px.inputs[px.pos]
            chars = tochars(chars, ranges)
            if chars.find(c) != -1:
                px.pos += 1
                return True
        return False
    return match